In [2]:
#### Required imports
import tensorflow_hub as hub
import tensorflow as tf
import os as os
import regex as re
import pandas as pd
import numpy as np
from transformers import BertTokenizer,BertConfig
from transformers import TFBertModel
from keras.models import Model
from keras import optimizers
from keras.metrics import MeanSquaredError
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [11]:
###### FinBERT Layer
config = BertConfig.from_pretrained(vocab_path='10k-sample/FinVocab-Uncased.txt',pretrained_model_name_or_path='10k-sample//config.json')
FinBERT_model = TFBertModel.from_pretrained(config=config,pretrained_model_name_or_path='10k-sample/pytorch_model.bin',from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [5]:
######## Function to extract the input text from the files ########
def process_inp_doc(path_file) :

  file_text = open(path_file,encoding='utf8').read()

  # remove punctations and digits and remove <PAGE> which was used for page number
  file_data = re.sub(r'[\d$%-:;!]', '', file_text)
  file_data = re.sub(r'<PAGE>', '', file_data)
  file_data = ''.join(file_data)

  return file_data

######## Function to extract the output values from the file ########
def process_out(company_id,output_file):
  
  with open(output_file,'r', encoding='utf-8') as m_file :
    for line in m_file.readlines():
      if company_id == line.split()[1]:
        return line.split()[0]
    print("not found")
  return None

######## Function to pre-process the documents from meta-file of a given year ########
def pre_processing(meta_file,output_file):
  
  with open(meta_file,'r', encoding='utf-8') as m_file :
    
    year = meta_file.split('/')[2].split('.')[0]
    dir_path = os.path.dirname(meta_file).split('/')[0] + '/all.tok/' +year+'.tok'
    data =[]
    
    for line in m_file.readlines():
      inp_path_file = dir_path +'/'+ line.split()[0] + '.mda'

      # get input sentences from the company document
      inp_sentences = process_inp_doc(inp_path_file)
    
      # get output value for the company
      out_values = float(process_out(line.split()[0],output_file))

      #insert values into the data list
      data.append({'text':inp_sentences,'value':out_values})

  return data

In [6]:
######## Function to get the encoded values ######## 
def FinBERT_encode(sentences, tokenizer, MAX_SEQ_LEN=512):

  all_tokens = []
  all_masks = []
  all_segments = []
  for sentence in sentences:
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[-MAX_SEQ_LEN+2:]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    token_ids = tokenizer.convert_tokens_to_ids(stokens,)

    ids = token_ids + [0] * (MAX_SEQ_LEN-len(token_ids))
    masks = [1]*len(token_ids) + [0] * (MAX_SEQ_LEN - len(token_ids))
    segments = [0] * (MAX_SEQ_LEN)

    all_tokens.append(ids)
    all_masks.append(masks)
    all_segments.append(segments)

  return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [7]:
##### function that defines the model
def get_model():

  input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="segment_ids")

  custom_objects={'leaky_relu': tf.nn.leaky_relu}

  model_output = FinBERT_model(input_word_ids, input_mask, segment_ids)
  clf_output = model_output.last_hidden_state
      
  net = tf.keras.layers.GlobalMaxPool1D()(clf_output)
  net = tf.keras.layers.Dense(1, activation='linear')(net)
  out = tf.keras.layers.Dense(1, activation='linear', name='output')(net)

  model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

  opt = optimizers.Adam(learning_rate=0.05)
  model.compile(optimizer=opt, loss='mse')

  return model

In [12]:
#### model summary 
MAX_SEQ_LEN = 512
model = get_model()
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]

In [13]:
##### Data extraction and Fitting the model
test_loss_all_years = []
train_loss_all_years = []
val_loss_all_years = []
history_all_years = []
data = []
n_splits = 5
epochs = 5
for year in range(2008,2014):
    
    ######## extracting text and storing it in dataframes ########
#     with tf.device('/device:GPU:0'):
    data_train = pre_processing('10k-sample/all.meta/'+str(year-3)+'.meta.txt','10k-sample/all.logfama/'+str(year-3)+'.logfama.txt')
    data_train.extend(pre_processing('10k-sample/all.meta/'+str(year-2)+'.meta.txt','10k-sample/all.logfama/'+str(year-2)+'.logfama.txt'))
    data_train.extend(pre_processing('10k-sample/all.meta/'+str(year-1)+'.meta.txt','10k-sample/all.logfama/'+str(year-1)+'.logfama.txt'))
    train_df = pd.DataFrame(data_train,columns=['text','value'])

    data_test = pre_processing('10k-sample/all.meta/'+str(year)+'.meta.txt','10k-sample/all.logfama/'+str(year)+'.logfama.txt')
    test_df = pd.DataFrame(data_test,columns=['text','value'])
    data.append({'year':year,'train_df_length':len(data_train),'test_df_length':len(data_test)})
    
    ###### removing few documents which are not processed properly####
    train_df = train_df.loc[train_df["text"].apply(lambda x: x.split().__len__())>256]
    test_df = test_df.loc[test_df["text"].apply(lambda x: x.split().__len__())>256]
    
    vocab_path = '10k-sample/FinVocab-Uncased.txt'
    ######## extracting tokens from dataframes ########

    tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

#     with tf.device('/device:GPU:0'):

    #### training 
    # input encoding
    sentences = train_df.text.values
    FinBERT_train_input = FinBERT_encode(sentences, tokenizer, MAX_SEQ_LEN)
    # output values
    FinBERT_train_output = train_df.value.values

    #### test
    # input encoding
    sentences = test_df.text.values
    FinBERT_test_input = FinBERT_encode(sentences, tokenizer, MAX_SEQ_LEN)
    # output values
    FinBERT_test_output = test_df.value.values
    
    ### applying minmax scalar
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    FinBERT_train_output = np.array(FinBERT_train_output).reshape(len(FinBERT_train_output),1)
    FinBERT_test_output = np.array(FinBERT_test_output).reshape(len(FinBERT_test_output),1)
    output = np.concatenate((FinBERT_train_output, FinBERT_test_output))
    output = scaler.fit_transform(output)
    FinBERT_train_output = output[:len(FinBERT_train_input[0])]
    FinBERT_test_output = output[-len(FinBERT_test_input[0]):]
    
    ######## Kfold training and saving checkpoints ########
#     with tf.device('/device:GPU:0'):
    kf = KFold(n_splits=n_splits)
    history =[]
    train_loss=[]
    vald_loss=[]
    test_loss = []
    fold = 1

    for train_index, test_index in kf.split(FinBERT_train_input[0]):

        checkpoint_filepath = 'Results/FinBERT_results_min/CheckPoints/'+str(year)+'FinBERT_checkpoint'+str(fold)
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                                                        filepath=checkpoint_filepath,
                                                                        save_weights_only=False,
                                                                        monitor='loss',
                                                                        mode='min',
                                                                        save_best_only=True
                                                                    )

        train_history = model.fit(
                                  [FinBERT_train_input[0][train_index],FinBERT_train_input[1][train_index],FinBERT_train_input[2][train_index]],#input
                                  FinBERT_train_output[train_index],#output
                                  epochs=epochs, #epochs
                                  verbose=1 ,
                                  batch_size = batch_size,
                                  callbacks=[model_checkpoint_callback]
                              )
        model_best = get_model()
        model_best.load_weights(checkpoint_filepath)

        fold+=1
        loss_T = model_best.evaluate([FinBERT_train_input[0][train_index],FinBERT_train_input[1][train_index],FinBERT_train_input[2][train_index]]
                                           , FinBERT_train_output[train_index], verbose=0)
        loss_V = model_best.evaluate([FinBERT_train_input[0][test_index],FinBERT_train_input[1][test_index],FinBERT_train_input[2][test_index]]
                                          , FinBERT_train_output[test_index], verbose=0)
        loss_test = model_best.evaluate([FinBERT_test_input[0],FinBERT_test_input[1],FinBERT_test_input[2]]
                                          , FinBERT_test_output, verbose=0)
        
        train_loss.append(loss_T)
        vald_loss.append(loss_V)
        history.append(train_history)
        test_loss.append(loss_test)
            
    test_loss_all_years.append(test_loss)
    train_loss_all_years.append(train_loss)
    val_loss_all_years.append(vald_loss)
    history_all_years.append(history)

FileNotFoundError: [Errno 2] No such file or directory: '10k-sample/all.meta/2005.meta.txt'

In [ ]:
stats_df = pd.DataFrame(data,columns=['year','train_df_length','test_df_length'])
stats_df.to_csv('Loss_values/BERT_stats_minmax.csv', header=False, index=False)

In [ ]:
import matplotlib.pylab as plt
data=[test_loss]
  
fig = plt.figure()  
# Creating axes instance 
ax = fig.add_axes([0, 0, 1, 1]) 
  
# Creating plot 
ax.boxplot(data)
years = [year for year in range(2008,2014)]
ax.set_xticklabels([year for year in range(2008,2014)]) 

# naming the y axis 
plt.ylabel('MSE Loss')
plt.title("Box plot for Test Loss")
textstr ='Test Loss for FinBERT : '+str(np.round(np.mean(test_loss),3))+' ('+str(np.round(np.std(test_loss),3))+')'
plt.gcf().text(0, -0.25, textstr, fontsize=14)
# show plot 
plt.savefig('Plots/block_plot_FinBERT_minmax.png',bbox_inches='tight')

In [ ]:
loss_data = []
train_data = []
vald_data = []
for year_loss_test,year_loss_train,year_loss_vald,year in zip(test_loss_all_years,train_loss_all_years,val_loss_all_years,years) :
    loss_data.append({'year':year,'value':year_loss_test})
    train_data.append({'year':year,'value':year_loss_train})
    vald_data.append({'year':year,'value':year_loss_vald})
    
loss_data_test_df = pd.DataFrame(loss_data,columns=['year','value'])
loss_data_test_df.to_csv('Loss_values/FinBERT_Loss_test_minmax.csv', header=False, index=False)

loss_data_train_df = pd.DataFrame(train_data,columns=['year','value'])
loss_data_train_df.to_csv('Loss_values/FinBERT_Loss_train_minmax.csv', header=False, index=False)

loss_data_vald_df = pd.DataFrame(vald_data,columns=['year','value'])
loss_data_vald_df.to_csv('Loss_values/FinBERT_Loss_vald_minmax.csv', header=False, index=False)